# **GIẢ LẬP KIẾN TRÚC RANC VỚI TẬP MNIST**


Tiến hành download toàn bộ code trong đường link này:
https://github.com/UA-RCL/RANC/tree/master/software
sau đó up lên colab, giải nén

Có thể sử dụng link này để download: https://minhaskamal.github.io/DownGit/#/home


In [ ]:
!unzip "software.zip"

Code này sẽ sử dụng tealayer2.0 (sử dụng tensorflow 2.x)

Sau khi giải nén, do code mặc định của họ đang sử dụng tensorflow 2.0.0b1 mà phiên bản đó giờ không còn được hỗ trợ nên cần phải chỉnh lại.

=> Truy cập "./sofware/tealayers/tealayer2.0/setup.py", chỉnh dòng 19 thành 'tensorflow-gpu==2.7.0' hoặc bất cứ phiên bản 2.x nào đang được hỗ trợ rồi lưu lại

Code của họ bị sai ở một đoạn, để sửa:

=> Đi đến "./software/tealayers/tealayer2.0/tealayer2/additivepooling.py"

Sửa dòng 73 thành "output = tf.reshape(output, [-1, int(self.num_inputs//self.num_classes), self.num_classes])" rồi lưu lại

In [ ]:
%cd "./software"
!pip install "./tealayers/tealayer2.0"
!pip install "./rancutils"

# **CÀI ĐẶT CÁC THƯ VIỆN CẦN THIẾT**

In [ ]:
from tealayer2 import Tea, AdditivePooling
from tensorflow.keras.layers import Flatten, Activation, Input, Lambda, concatenate
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


# **KHỞI TẠO MẠNG**

In [ ]:
# Greyscale images are of shape (28,28,1)
inputs = Input(shape=(28,28,1))

# Flatten the inputs so that inputs map as: flatten_input[0] -> axon[0], ..., flatten_input[255] -> axon[255]
flattened_inputs = Flatten()(inputs)

# Generate each core.
# We are taking a 16x16 square of the input image and striding it by 12. this gives us 4 cores with 0 padding encumpassing the entire image.
# Trải phẳng các ảnh input 28x28x1 thành 1 vector 784x1, quét từng đoạt 256 để đưa vào lần lượt các core, mỗi đoạn sau stride 12 phần tử so với đoạn trước
# (Trong chương 02 ở file doc là quét 4 góc bức ảnh, nó cũng tương tự như thế này)
core0 = Lambda(lambda x : x[:, :256])(flattened_inputs)
core1 = Lambda(lambda x : x[:, 176:432])(flattened_inputs)
core2 = Lambda(lambda x : x[:, 352:608])(flattened_inputs)
core3 = Lambda(lambda x : x[:, 528:])(flattened_inputs)

# Use the image distributions as corresponding inputs into our Tea Layer.
# units là số neuron được sử dụng cho tea layer này, ở đây khai báo 4 tea layer với units = 64 (tương ứng 4 core với mỗi core sử dụng 64 neuron)
core0 = Tea(units=64, name='tea_1_1')(core0)
core1 = Tea(units=64, name='tea_1_2')(core1)
core2 = Tea(units=64, name='tea_1_3')(core2)
core3 = Tea(units=64, name='tea_1_4')(core3)

# The classification is the concatenation of these 4 core's outputs.
# We'll call the classification core our 'network'
network = concatenate([core0, core1, core2, core3])

# Gộp đầu ra của 4 core trên lại thành 1 layer khác (tổng sẽ là 256 đầu vào cho tea layer này), tuy nhiên
# do chỉ có 10 class (các số từ 0 đến 9, mỗi số 1 class) => layer này chỉ sử dụng 250 neuron (do 256 không
# chia hết cho 10), trong đó cứ 25 neuron thì sẽ vote cho 1 class, số lượng neuron vote cho class nào lớn 
# nhất thì ảnh sẽ thuộc về class đó. Ví dụ trong 250 đầu ra, từ 0 đến 24 có 17 spike bắn ra, đồng thời 
# không có cụm nào bắn ra được nhiều bằng hoặc hơn 17 => Ảnh thuộc về class số "0" 
network = Tea(units=250, name='tea_2')(network)

network = AdditivePooling(10)(network)

# **TRAIN MODEL**

In [ ]:
# Chuẩn bị dataset để train
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
print(y_test.shape)


predictions = Activation('softmax')(network)

model = Model(inputs=inputs, outputs=predictions)

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_split=0.2)

score = model.evaluate(X_test, y_test, verbose=0)

print("Test Loss: ", score[0])
print("Test Accuracy: ", score[1])

48000/48000 [==============================] - 2s 43us/sample - loss: 0.2947 - acc: 0.9088 - val_loss: 0.3652 - val_acc: 0.8879
Epoch 3/10
48000/48000 [==============================] - 2s 43us/sample - loss: 0.2338 - acc: 0.9277 - val_loss: 0.2981 - val_acc: 0.9027
Epoch 4/10
48000/48000 [==============================] - 2s 44us/sample - loss: 0.2011 - acc: 0.9380 - val_loss: 0.2971 - val_acc: 0.9016
Epoch 5/10
48000/48000 [==============================] - 2s 44us/sample - loss: 0.1801 - acc: 0.9435 - val_loss: 0.2728 - val_acc: 0.9109
Epoch 6/10
48000/48000 [==============================] - 2s 45us/sample - loss: 0.1710 - acc: 0.9460 - val_loss: 0.2329 - val_acc: 0.9237
Epoch 7/10
48000/48000 [==============================] - 2s 44us/sample - loss: 0.1538 - acc: 0.9504 - val_loss: 0.2431 - val_acc: 0.9210
Epoch 8/10
48000/48000 [==============================] - 2s 44us/sample - loss: 0.1438 - acc: 0.9545 - val_loss: 0.2261 - val_acc: 0.9283
Epoch 9/10
48000/48000 [==============

# **GENERATE MẠNG VÀ CÁC PACKET**

In [ ]:
# Optionally, then save the generated network out for use in the simulator and/or hardware
from rancutils.teaconversion import create_cores, create_packets, Packet
from rancutils.output_bus import OutputBus
from rancutils.serialization import save as sim_save

x_test_flat = X_test.reshape((10000, 784))
partitioned_packets = []

#test với 10000 ảnh
num_test_samples = 10
# Tạo các core bằng hàm create_cores(), sử dụng 2 layer (1 layer 4 core và 1 layer 1 core), sử dụng absolute reset mode: neuron_reset_type=0
cores_sim = create_cores(model, 2, neuron_reset_type=0) 
# Partition the packets into groups as they will be fed into each of the input cores
partitioned_packets.append(x_test_flat[:num_test_samples, :256])
partitioned_packets.append(x_test_flat[:num_test_samples, 176:432])
partitioned_packets.append(x_test_flat[:num_test_samples, 352:608])
partitioned_packets.append(x_test_flat[:num_test_samples, 528:])
# Tạo packet bằng hàm create_packets()
packets_sim = create_packets(partitioned_packets)
# Tạo output_bus bằng hàm OutputBus(coordinate, num_outputs)
output_bus_sim = OutputBus((0, 2), num_outputs=250)

# Đây chính là file đầu vào cho giả lập kiến trúc RANC bằng code C++
sim_save("mnist_config.json", cores_sim, packets_sim, output_bus_sim, indent=2)
# Lưu lại đầu ra của tensorflow predictions và correct labels để tý làm cross validation
predict = model.predict(X_test[:num_test_samples,:])
idx = []
for i in predict:
  idx.append(np.argmax(i))
test_predictions = to_categorical(idx)
np.save("mnist_tf_preds.txt", test_predictions)
np.save("mnist_correct_preds.txt", y_test[:num_test_samples,:])

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


**Lưu file mem**

In [ ]:
from rancutils.emulation import output_for_testbench, output_for_streaming

In [ ]:
output_for_streaming(cores_sim,max_xy=(4,2),output_path="/content/mnist_5_core_mem")

In [ ]:
!zip -r /content/mnist_5_core_mem.zip /content/mnist_5_core_mem

In [ ]:
from google.colab import files
files.download("/content/mnist_5_core_mem.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Lưu input và output chuẩn**

In [ ]:
# lưu input và output chuẩn
output_for_testbench(packets_sim,
                         y_test[:num_test_samples,:],
                         output_path='/content/',
                         input_filename='tb_input.txt',
                         correct_filename='tb_correct.txt',
                         num_inputs_filename='tb_num_inputs.txt',
                         num_outputs_filename='tb_num_outputs.txt',
                         max_packet_xy=(512, 512),
                         num_axons=256,
                         num_ticks=16,
                         num_outputs=250)

Sau khi chạy xong đoạn code này, 3 file mới sẽ được tạo ra. Tiến hành tải file "mnist_config.json" về để làm input file cho code RANC simulator C++

# **C++**

**Compile code**

In [ ]:
cd "/content/"

/content


In [ ]:
!unzip "simulator.zip"

In [ ]:
cd "./simulator/"

/content/simulator


In [ ]:
!mkdir build

In [ ]:
cd build

/content/simulator/build


In [ ]:
!cmake ".."
!make

**Chạy code**

Muốn chạy với x ảnh thì số tick là x + 2, do trễ 2 tick từ 2 layer

In [ ]:
!/content/simulator/build/ranc_sim -i /content/software/mnist_config.json -o /content/simulator_output.txt -c /content/simulator/config.json 12

# **SO SÁNH ĐẦU RA TỪ SIMULATOR VỚI ĐẦU RA CỦA TENSORFLOW**

Tiến hành upload file output của simulator lên colab, chạy đoạn code này để so sánh

Lưu ý:

- Tên file đặt là "simulator_output.txt" và lưu trong đường dẫn hiện tại đang cd đến

- Trước khi upload, xóa 2 dòng đầu tiên trong file output đi (chi tiết trong file doc)

In [ ]:
    # TODO: Add usage example for outputting to emulation via rancutils.emulation.write_cores, etc.

    ###
    # ...
    # Start the simulator, etc, and collect results...
    # ...
    ###

    # Process the output to collect final classificaiton results and compare against Tensorflow predictions
    from rancutils.simulator import collect_classifications_from_simulator
    tf_output = np.load("/content/software/mnist_tf_preds.txt"+".npy")
    correct_output = np.load("/content/software/mnist_correct_preds.txt"+".npy")
    simulator_output = collect_classifications_from_simulator("/content/simulator_output.txt", num_classes=10)
    
    tf_output_flat = np.array([0] * tf_output.shape[0])
    for i in range(tf_output.shape[0]):
        output_i = tf_output[i, :]
        decision = np.where(output_i == max(output_i))[0]
        if len(decision) > 1:
            decision = decision[0]
        tf_output_flat[i] = decision
    tf_output = tf_output_flat
    
    correct_output_flat = np.array([0] * correct_output.shape[0])
    for i in range(correct_output.shape[0]):
        output_i = correct_output[i, :]
        decision = np.where(output_i == max(output_i))[0]
        if len(decision) > 1:
            decision = decision[0]
        correct_output_flat[i] = decision
    correct_output = correct_output_flat

    if all(tf_output == simulator_output):
        print("Tensorflow output matches simulator output exactly!")
        print(f"Testing accuracy against known class labels is {(len(np.where(tf_output == correct_output[:len(tf_output)])[0]) / len(tf_output)) * 100}%")
    else:
        print("There are differences between Tensorflow and the simulator...")
        print(f"Differences are in indices {np.where(tf_output != simulator_output)}")
        print(f"Tensorflow thought the classes were {tf_output[np.where(tf_output != simulator_output)]}")
        print(f"The simulator thought the classes were {simulator_output[np.where(tf_output != simulator_output)]}")
        print(f"The correct classes were {correct_output[np.where(tf_output != simulator_output)]}")

Tensorflow output matches simulator output exactly!
Testing accuracy against known class labels is 90.0%
